# Simplified Spatial Pooler

Here I am trying to get used to Julia by implementing Spatial Pooler w/o any fancy features. Just plain compute with learning.

In [5]:
struct Sds
    shape::Tuple{Int, Vararg{Int}}
    size::Int
    sparsity::Float64
    active_size::Int
    
    Sds(size::Int, sparsity::Float64) = new(
        (size,), size, sparsity, 
        round(Int, size * sparsity)
    )
    
    Sds(size::Int, active_size::Int) = new(
        (size,), size, active_size / size, active_size
    )
end

In [6]:
@assert Sds(100, .3).active_size == 30
@assert Sds(80, 10).sparsity == 0.125

In [7]:
sds = Sds(20, 4)

Sds((20,), 20, 0.2, 4)

In [8]:
using Random

seed = 42
rng = MersenneTwister(seed)

MersenneTwister(42)

In [9]:
using Pkg
Pkg.add("StatsBase")
Pkg.add("Distributions")
Pkg.add("BenchmarkTools")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [10]:
using StatsBase
using Distributions
using BenchmarkTools

In [11]:
seed = 42
rng = MersenneTwister(seed)

sample(rng, 1:100, 10, replace=false, ordered=true)

10-element Vector{Int64}:
  6
 14
 47
 59
 60
 61
 71
 84
 85
 94

In [12]:
@benchmark sample(rng, 1:100, 10, replace=false, ordered=true)

BenchmarkTools.Trial: 10000 samples with 326 evaluations.
 Range (min … max):  278.500 ns …  1.397 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     283.230 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   285.923 ns ± 35.848 ns  ┊ GC (mean ± σ):  0.37% ± 2.48%

        ▁█▇▇▇▂▂                                                 
  ▁▁▁▂▃▇████████▇▆▄▄▂▂▂▂▂▂▂▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  278 ns          Histogram: frequency by time          305 ns <

 Memory estimate: 144 bytes, allocs estimate: 1.

In [13]:
function rand_sparse(rng::AbstractRNG, sds::Sds)::Vector{Int}
    return sample(rng, 1:sds.size, sds.active_size, replace=false, ordered=true)
end

function sparse_to_dense(sparse_sdr::Vector{Int}, sds::Sds)::Vector{Bool}
    a = zeros(Bool, sds.size)
    a[sparse_sdr] .= 1
    return a
end

rand_sparse(rng, sds), sparse_to_dense(rand_sparse(rng, sds), sds)

([5, 8, 10, 16], Bool[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0])

In [14]:
sds_in, sds_out = Sds(20, 5), Sds(30, 4)
perm_threshold = 0.5

weights = rand(sds_in.size, sds_out.size)
synapses = weights .>= perm_threshold

synapses

20×30 BitMatrix:
 0  1  1  1  0  0  1  0  0  1  1  1  1  …  1  0  0  1  1  1  0  0  1  0  0  0
 0  0  1  0  0  0  1  1  0  0  0  1  0     1  0  0  1  1  0  1  0  0  0  0  1
 1  1  0  0  1  0  0  1  1  1  1  0  1     0  1  1  1  0  1  1  0  1  1  1  1
 1  0  1  0  0  1  1  1  1  1  0  1  0     1  0  0  0  1  0  0  0  0  1  0  0
 0  0  0  1  0  1  0  0  0  1  1  1  0     1  0  0  0  1  1  1  0  0  1  1  0
 1  1  1  0  1  1  0  0  0  0  1  1  1  …  0  0  1  0  1  1  0  1  0  0  0  1
 1  0  0  0  1  0  1  0  0  0  1  0  0     1  1  1  1  0  0  0  1  1  0  1  1
 1  0  1  0  1  0  0  1  1  0  0  0  1     0  0  1  0  0  0  0  0  0  1  1  0
 1  0  1  1  0  0  0  0  0  0  0  0  0     1  0  1  1  1  0  0  1  1  1  0  0
 1  0  1  0  0  0  1  0  0  1  0  0  0     1  1  1  1  1  0  0  1  1  0  0  1
 0  0  1  0  1  0  1  1  1  0  0  1  0  …  0  1  0  1  1  0  1  0  0  0  1  1
 0  0  0  1  1  0  0  0  0  1  0  1  0     1  0  0  0  0  1  0  1  0  0  0  0
 0  1  1  1  1  1  1  0  1  0  0  0  1     1  0

In [15]:
input_sdr = rand_sparse(rng, sds_in)

synapses[input_sdr, :]

5×30 BitMatrix:
 1  1  0  0  1  0  0  1  1  1  1  0  1  …  0  1  1  1  0  1  1  0  1  1  1  1
 0  0  0  1  0  1  0  0  0  1  1  1  0     1  0  0  0  1  1  1  0  0  1  1  0
 1  0  0  0  1  0  1  0  0  0  1  0  0     1  1  1  1  0  0  0  1  1  0  1  1
 1  1  1  1  1  1  1  0  0  0  1  1  0     0  0  0  1  0  1  1  0  1  1  0  0
 1  1  0  0  0  0  1  1  1  1  1  1  1     1  0  1  0  0  0  0  1  0  0  1  0

In [16]:
overlaps = dropdims(sum(synapses[input_sdr, :], dims=1), dims=1)

30-element Vector{Int64}:
 4
 3
 1
 2
 3
 2
 3
 2
 2
 3
 5
 3
 2
 ⋮
 3
 2
 3
 3
 1
 3
 3
 2
 3
 3
 4
 2

In [17]:
winners = partialsortperm(overlaps, 1:sds_out.active_size, rev=true)

4-element view(::Vector{Int64}, 1:4) with eltype Int64:
 11
  1
 14
 29

In [18]:
overlaps[winners]

4-element Vector{Int64}:
 5
 4
 4
 4

In [25]:
function compute_overlaps(input_sdr::Vector{Int}, synapses::BitMatrix, sds_out)::Vector{Int}
    overlaps = dropdims(sum(synapses[input_sdr, :], dims=1), dims=1)
    return overlaps
end

function compute_winners(overlaps::Vector{Int}, sds_out)::Vector{Int}
    winners = partialsortperm(overlaps, 1:sds_out.active_size, rev=true)
    return winners
end

function update_permanence(weights, winners, (perm_inc, perm_dec))
    weights[:, winners] .-= perm_dec
    weights[input_sdr, winners] .+= perm_inc

    clamp!(weights[:, winners], 0., 1.)
end

update_permanence (generic function with 1 method)

In [26]:
@benchmark compute_winners(compute_overlaps(rand_sparse(rng, sds_in), synapses, sds_out), sds_out)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.083 μs … 168.312 μs  ┊ GC (min … max): 0.00% … 98.86%
 Time  (median):     1.217 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.266 μs ±   2.361 μs  ┊ GC (mean ± σ):  2.63% ±  1.40%

           ▁▃▅▆███▇▅▅▃▂▁                                       
  ▂▂▂▂▃▄▄▆▇███████████████▇▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂ ▄
  1.08 μs         Histogram: frequency by time        1.57 μs <

 Memory estimate: 1.03 KiB, allocs estimate: 10.

In [33]:
@benchmark begin

perm_inc, perm_dec = 0.1, 0.01

input_sdr = rand_sparse(rng, sds_in)
for _ in 1:10
    overlaps = compute_overlaps(input_sdr, $synapses, sds_out)
    winners = compute_winners(overlaps, sds_out)
    update_permanence($weights, winners, (perm_inc, perm_dec))
    $synapses = weights .>= perm_threshold
end
end

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  22.791 μs …  2.342 ms  ┊ GC (min … max): 0.00% … 98.24%
 Time  (median):     26.916 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   29.836 μs ± 79.385 μs  ┊ GC (mean ± σ):  9.52% ±  3.53%

           ▂▂     ▁▁▅▇▇█▇▄▅▃▂                                  
  ▂▂▂▃▄▄▅▇███▇██▇█████████████▆▄▄▅▅▄▄▃▄▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▂ ▄
  22.8 μs         Histogram: frequency by time        34.7 μs <

 Memory estimate: 72.75 KiB, allocs estimate: 251.